In [ ]:
# ! pip install tensorflow
# ! pip install pandas
# ! pip install openpyxl
! pip install matplotlib==3.7.2

In [10]:
# ! pip uninstall tensorflow
# ! pip install pandas
# ! pip install openpyxl
#! pip install fsspec
# pip install Pillow
#! pip install tensorflow==2.2.0

! pip install h5py

#! pip install scipy

In [1]:

import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers
from tensorflow.keras import Model


# Define paths
data_dir = "F:/ML FInal/6/train"
csv_path = "F:/ML FInal/6/train/0_classes.csv"

# Load Excel sheet
df = pd.read_csv(csv_path)
print(df.columns)

image_count = 0
for filename in os.listdir(data_dir):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):  # Adjust for relevant extensions
        image_count += 1

print("Number of images in " + data_dir + ": ", image_count)




Index(['filename', ' Apple', ' Banana', ' Grapes', ' Kiwi', ' Mango',
       ' Orange', ' Pineapple', ' Sugerapple', ' Watermelon'],
      dtype='object')
Number of images in F:/ML FInal/6/train:  2697


In [2]:
# Get a list of all columns
all_columns = df.columns.tolist()

# Remove the filename column
target_columns = [col for col in all_columns if col != 'filename']






In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import scipy
#from keras.preprocessing import image
# Create an instance of the ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Generate the training and validation data
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=data_dir,
    x_col="filename",
    y_col=target_columns,
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(224, 224)
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=data_dir,
    x_col="filename",
    y_col=target_columns,
    subset="validation",
    batch_size=32,

    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(224, 224)
)

# Load the MobileNetV2 model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.2)(x)
predictions = layers.Dense(len(target_columns), activation="sigmoid")(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // valid_generator.batch_size,
    epochs=10
)



Found 2158 validated image filenames.
Found 539 validated image filenames.
Epoch 1/10
67/67 [==============================] - 55s 781ms/step - loss: 0.1228 - accuracy: 0.8269 - val_loss: 0.0402 - val_accuracy: 0.9746
Epoch 2/10
67/67 [==============================] - 46s 690ms/step - loss: 0.0349 - accuracy: 0.9675 - val_loss: 0.0275 - val_accuracy: 0.9785
Epoch 3/10
67/67 [==============================] - 45s 676ms/step - loss: 0.0201 - accuracy: 0.9835 - val_loss: 0.0236 - val_accuracy: 0.9824
Epoch 4/10
67/67 [==============================] - 49s 733ms/step - loss: 0.0129 - accuracy: 0.9915 - val_loss: 0.0183 - val_accuracy: 0.9824
Epoch 5/10
67/67 [==============================] - 45s 670ms/step - loss: 0.0087 - accuracy: 0.9949 - val_loss: 0.0167 - val_accuracy: 0.9824
Epoch 6/10
67/67 [==============================] - 45s 671ms/step - loss: 0.0074 - accuracy: 0.9944 - val_loss: 0.0159 - val_accuracy: 0.9824
Epoch 7/10
67/67 [==============================] - 46s 685ms/step 

In [8]:


# Define paths
test_dir = "F:/ML FInal/6/test"
test_csv_path = "F:/ML FInal/6/test/0_classes.csv"

# Load Excel sheet
df_test_dir = pd.read_csv(test_csv_path)
print(df_test_dir.columns)

image_count = 0
for filename in os.listdir(test_dir):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):  # Adjust for relevant extensions
        image_count += 1

print("Number of images in " + test_dir  + ": ", image_count)


Index(['filename', ' Apple', ' Banana', ' Grapes', ' Kiwi', ' Mango',
       ' Orange', ' Pineapple', ' Sugerapple', ' Watermelon'],
      dtype='object')
Number of images in F:/ML FInal/6/test:  90


In [9]:
# Generate the test data
test_generator = datagen.flow_from_dataframe(
    dataframe=df_test_dir,
    directory=test_dir,
    x_col="filename",
    y_col=target_columns,
    batch_size=32,
    seed=42,
    shuffle=False,  # Don't shuffle test data
    class_mode="raw",
    target_size=(224, 224)
)

# Test the model
loss, accuracy = model.evaluate(test_generator)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Found 90 validated image filenames.
3/3 [==============================] - 2s 488ms/step - loss: 0.0287 - accuracy: 0.9667
Test Loss: 0.028693431988358498
Test Accuracy: 0.9666666388511658


In [12]:
from keras.models import load_model

# returns a compiled model identical to the previous one
model = load_model('assesment4_group6.h5')

In [11]:
model.save('assesment4_group6.h5')  # creates a HDF5 file 'my_model.h5'

c:\Users\ACER\miniconda3\envs\assesment\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
import os
import tensorflow as tf
from tensorflow.keras.models import save_model

# Ensure the directory exists
save_directory = 'F:\mlModel'
os.makedirs(save_directory, exist_ok=True)

# Save the model using tf.keras.models.save_model
save_model(model, save_directory)

INFO:tensorflow:Assets written to: F:\mlModel\assets


INFO:tensorflow:Assets written to: F:\mlModel\assets
